In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import thu vien 
import pandas as pd
import numpy as np
# import mo hinh
from sklearn.linear_model import LogisticRegression

# # import model selection - dung de phan chia du lieu tran/test
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# import feature extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# Import metric danh gia
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, precision_score, recall_score

import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('drive/My Drive/Bitcoin_dataset_updated.csv', index_col=False)
data_prime = data.copy()
data_prime

,Date,BTC price [USD],n-transactions,fee [USD],btc search trends,Gold price[USD],SP500 close index,Oil WTI price[USD],M2(Not seasonally adjusted)[1e+09 USD]
0,2014-01-02,749.27,54770.0,0.18,8,1225.0,1831.98,95.14,11089.3
1,2014-01-03,781.23,60980.0,NaN,8,1238.4,1831.37,93.66,11089.3
2,2014-01-04,807.39,52052.0,NaN,8,NaN,NaN,NaN,11089.3
3,2014-01-05,828.74,58662.0,0.38,8,NaN,NaN,NaN,11089.3
4,2014-01-06,904.98,67358.0,NaN,8,1237.8,1826.77,93.12,11089.3
...,...,...,...,...,...,...,...,...,...
2308,2020-04-25,7507.08,280805.0,0.75,16,NaN,NaN,NaN,17269.8
2309,2020-04-26,7549.52,252833.0,0.62,16,NaN,NaN,NaN,17269.8
2310,2020-04-27,7699.27,311217.0,0.69,16,1723.8,2878.48,12.17,17269.8
2311,2020-04-28,7790.66,323475.0,0.66,16,1722.2,2863.39,12.40,17269.8


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313 entries, 0 to 2312
Data columns (total 9 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Date                                    2313 non-null   object 
 1   BTC price [USD]                         2313 non-null   float64
 2   n-transactions                          2313 non-null   float64
 3   fee [USD]                               1469 non-null   float64
 4   btc search trends                       2313 non-null   int64  
 5   Gold price[USD]                         1671 non-null   float64
 6   SP500 close index                       1592 non-null   float64
 7   Oil WTI price[USD]                      1587 non-null   float64
 8   M2(Not seasonally adjusted)[1e+09 USD]  2313 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 162.8+ KB


In [ ]:
## Cột  fee [USD], Gold price[USD], SP500 cose index, oil WTI price[USD] bị mất giá trị (giá trị = NaN)

In [ ]:
print("---So diem du lieu--- "+ '\nso dong: ' +str(data.shape[0])+'\nSo cot: ' + str(data.shape[1]))

---So diem du lieu--- 
so dong: 2313
So cot: 9


In [ ]:
data.describe()

,BTC price [USD],n-transactions,fee [USD],btc search trends,Gold price[USD],SP500 close index,Oil WTI price[USD],M2(Not seasonally adjusted)[1e+09 USD]
count,2313.000000,2313.000000,1469.000000,2313.000000,1671.000000,1592.000000,1587.000000,2313.000000
mean,3616.144903,214401.246433,2.211995,11.176394,1337.838671,2397.414981,58.587738,13286.782447
std,3877.083340,96042.289290,5.573905,13.993820,111.052949,402.628295,18.809558,1350.449720
min,172.000000,47740.000000,0.030000,2.000000,1070.800000,1741.890000,-36.980000,11089.300000
25%,440.030000,121973.000000,0.220000,3.000000,1272.300000,2058.567500,47.460000,12058.700000
50%,1056.000000,224394.000000,0.670000,7.000000,1320.900000,2346.590000,53.860000,13467.300000
75%,6763.750000,291597.000000,1.690000,13.000000,1385.100000,2748.372500,64.535000,14209.700000
max,19279.900000,490644.000000,54.790000,100.000000,1768.900000,3386.150000,107.950000,17269.800000


In [ ]:
# Xem tóm tắt các thuộc tính dạng số
data.describe()

,BTC price [USD],n-transactions,fee [USD],btc search trends,Gold price[USD],SP500 close index,Oil WTI price[USD],M2(Not seasonally adjusted)[1e+09 USD]
count,2313.000000,2313.000000,1469.000000,2313.000000,1671.000000,1592.000000,1587.000000,2313.000000
mean,3616.144903,214401.246433,2.211995,11.176394,1337.838671,2397.414981,58.587738,13286.782447
std,3877.083340,96042.289290,5.573905,13.993820,111.052949,402.628295,18.809558,1350.449720
min,172.000000,47740.000000,0.030000,2.000000,1070.800000,1741.890000,-36.980000,11089.300000
25%,440.030000,121973.000000,0.220000,3.000000,1272.300000,2058.567500,47.460000,12058.700000
50%,1056.000000,224394.000000,0.670000,7.000000,1320.900000,2346.590000,53.860000,13467.300000
75%,6763.750000,291597.000000,1.690000,13.000000,1385.100000,2748.372500,64.535000,14209.700000
max,19279.900000,490644.000000,54.790000,100.000000,1768.900000,3386.150000,107.950000,17269.800000


In [ ]:
# Tính hệ số tương quan của median_house_value với các thuộc tính khác
corr_matrix = data.corr()
corr_matrix["BTC price [USD]"].sort_values(ascending=False)

BTC price [USD]                           1.000000
SP500 close index                         0.854566
M2(Not seasonally adjusted)[1e+09 USD]    0.756824
btc search trends                         0.731253
n-transactions                            0.611812
Gold price[USD]                           0.560585
fee [USD]                                 0.514952
Oil WTI price[USD]                       -0.041544
Name: BTC price [USD], dtype: float64

In [ ]:
# Lấy dữ liệu cột date ra
data = data.drop('Date',axis=1)
data_date = data_prime['Date']


In [ ]:
# Xử lí mất dữ liệu
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")              # SimpleImputer áp dụng transformer cho các giá trị bị mất (median : thay giá trị trung bình)
encoded = imputer.fit_transform(data)
encoded

array([[7.49270e+02, 5.47700e+04, 1.80000e-01, ..., 1.83198e+03,
        9.51400e+01, 1.10893e+04],
       [7.81230e+02, 6.09800e+04, 6.70000e-01, ..., 1.83137e+03,
        9.36600e+01, 1.10893e+04],
       [8.07390e+02, 5.20520e+04, 6.70000e-01, ..., 2.34659e+03,
        5.38600e+01, 1.10893e+04],
       ...,
       [7.69927e+03, 3.11217e+05, 6.90000e-01, ..., 2.87848e+03,
        1.21700e+01, 1.72698e+04],
       [7.79066e+03, 3.23475e+05, 6.60000e-01, ..., 2.86339e+03,
        1.24000e+01, 1.72698e+04],
       [7.76533e+03, 3.31401e+05, 1.30000e+00, ..., 2.93951e+03,
        1.50400e+01, 1.72698e+04]])

In [ ]:
data = pd.DataFrame(encoded,columns = data.columns)
data

,BTC price [USD],n-transactions,fee [USD],btc search trends,Gold price[USD],SP500 close index,Oil WTI price[USD],M2(Not seasonally adjusted)[1e+09 USD]
0,749.27,54770.0,0.18,8.0,1225.0,1831.98,95.14,11089.3
1,781.23,60980.0,0.67,8.0,1238.4,1831.37,93.66,11089.3
2,807.39,52052.0,0.67,8.0,1320.9,2346.59,53.86,11089.3
3,828.74,58662.0,0.38,8.0,1320.9,2346.59,53.86,11089.3
4,904.98,67358.0,0.67,8.0,1237.8,1826.77,93.12,11089.3
...,...,...,...,...,...,...,...,...
2308,7507.08,280805.0,0.75,16.0,1320.9,2346.59,53.86,17269.8
2309,7549.52,252833.0,0.62,16.0,1320.9,2346.59,53.86,17269.8
2310,7699.27,311217.0,0.69,16.0,1723.8,2878.48,12.17,17269.8
2311,7790.66,323475.0,0.66,16.0,1722.2,2863.39,12.40,17269.8


In [ ]:
out = data['BTC price [USD]']
data = data.drop('BTC price [USD]',axis=1)


In [ ]:
# Dữ liệu bị mất (NaN) đẫ được thay thế bằng giá trị trung bình

In [ ]:
# Số hóa dữ liệu bằng chuối bit nhị phân
from sklearn.preprocessing import LabelBinarizer
date_encoder = LabelBinarizer()
date_encoded = date_encoder.fit_transform(data_date)
date_encoded

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [ ]:
# from sklearn.base import BaseEstimator, TransformerMixin
# class DataFrameSelector(BaseEstimator, TransformerMixin):
#   def __init__(self, attribute_names):
#     self.attribute_names = attribute_names
#   def fit(self, X, y=None):
#     return self
#   def transform(self, X):
#     return X[self.attribute_names].values


In [ ]:
data1 = data.copy()
data1

,n-transactions,fee [USD],btc search trends,Gold price[USD],SP500 close index,Oil WTI price[USD],M2(Not seasonally adjusted)[1e+09 USD]
0,54770.0,0.18,8.0,1225.0,1831.98,95.14,11089.3
1,60980.0,0.67,8.0,1238.4,1831.37,93.66,11089.3
2,52052.0,0.67,8.0,1320.9,2346.59,53.86,11089.3
3,58662.0,0.38,8.0,1320.9,2346.59,53.86,11089.3
4,67358.0,0.67,8.0,1237.8,1826.77,93.12,11089.3
...,...,...,...,...,...,...,...
2308,280805.0,0.75,16.0,1320.9,2346.59,53.86,17269.8
2309,252833.0,0.62,16.0,1320.9,2346.59,53.86,17269.8
2310,311217.0,0.69,16.0,1723.8,2878.48,12.17,17269.8
2311,323475.0,0.66,16.0,1722.2,2863.39,12.40,17269.8


In [ ]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313 entries, 0 to 2312
Data columns (total 7 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   n-transactions                          2313 non-null   float64
 1   fee [USD]                               2313 non-null   float64
 2   btc search trends                       2313 non-null   float64
 3   Gold price[USD]                         2313 non-null   float64
 4   SP500 close index                       2313 non-null   float64
 5   Oil WTI price[USD]                      2313 non-null   float64
 6   M2(Not seasonally adjusted)[1e+09 USD]  2313 non-null   float64
dtypes: float64(7)
memory usage: 126.6 KB


In [ ]:
# Thêm giá trị date vào mảng dữ liệu
datarray = np.array(data1)
d1 =np.append(datarray,date_encoded,1)
d1

array([[5.47700e+04, 1.80000e-01, 8.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [6.09800e+04, 6.70000e-01, 8.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [5.20520e+04, 6.70000e-01, 8.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       ...,
       [3.11217e+05, 6.90000e-01, 1.60000e+01, ..., 1.00000e+00,
        0.00000e+00, 0.00000e+00],
       [3.23475e+05, 6.60000e-01, 1.60000e+01, ..., 0.00000e+00,
        1.00000e+00, 0.00000e+00],
       [3.31401e+05, 1.30000e+00, 1.60000e+01, ..., 0.00000e+00,
        0.00000e+00, 1.00000e+00]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(d1, out, random_state=0,test_size=0.25)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1734, 2317)
(1734,)
(579, 2317)
(579,)


In [ ]:
y_train

820      414.83
1185    1073.09
1591    9015.88
564      275.47
294      384.07
         ...   
1033     697.23
1731    6686.08
763      368.46
835      429.45
1653    6403.18
Name: BTC price [USD], Length: 1734, dtype: float64

In [ ]:
# Huấn luyện theo LinearRegression()
from sklearn.linear_model import LinearRegression
model_LinearR = LinearRegression()
mymodel_LinearR = model_LinearR.fit(X_train, y_train)
# Huấn luyện DecisionTree 
from sklearn.tree import DecisionTreeRegressor
model_DecisionT = DecisionTreeRegressor()
mymodel_DecisionT = model_DecisionT.fit(X_train,y_train)

In [ ]:
#do chinh xac DecisionTree

#RMSE DecisionTree
from sklearn.metrics import mean_squared_error
y_pred_DecisionT = mymodel_DecisionT.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_DecisionT)
tree_rmse = np.sqrt(tree_mse)
print("DecisionTree RMSE: "+str(tree_rmse))

print('Độ chính xác của thuật toán DecisionTree: '+str(mymodel_DecisionT.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_DecisionT = r2_score(y_test, y_pred_DecisionT)
print('Decision R2: '+ str(r2_DecisionT))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Decision = explained_variance_score(y_test, y_pred_DecisionT)
print('Decision explained variance: '+str(ev_Decision))

DecisionTree RMSE: 495.4496769203195
Độ chính xác của thuật toán DecisionTree: 0.9835312334605354
Decision R2: 0.9835312334605354
Decision explained variance: 0.9835751778435041


In [ ]:
#RMSE LinearGegression
from sklearn.metrics import mean_squared_error
y_pred_LinearR = mymodel_LinearR.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_LinearR)
Linear_rmse = np.sqrt(tree_mse)
print("Linear RMSE:" + str(Linear_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán LinearRegression: '+str(mymodel_LinearR.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_Linear = r2_score(y_test, y_pred_LinearR)
print('Linear R2: '+ str(r2_Linear))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Linear = explained_variance_score(y_test, y_pred_LinearR)
print('Linear explained variance: '+str(ev_Linear))
# 

Linear RMSE:1415.4267159088004
Độ chính xác của thuật toán LinearRegression: 0.8655883999218408
Linear R2: 0.8655883999218408
Linear explained variance: 0.8655892195469982


In [ ]:
#lasso model
from sklearn import linear_model
mymodel_lasso = linear_model.Lasso(alpha=0.1)
mymodel_lasso.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 263655442.26447216, tolerance: 2610633.987611766
  positive)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
#Đánh giá lasso model
from sklearn.metrics import mean_squared_error
y_pred_lasso = mymodel_lasso.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_LinearR)
lasso_rmse = np.sqrt(tree_mse)
print("Lasso RMSE:" + str(lasso_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán lasso: '+str(mymodel_lasso.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_lasso = r2_score(y_test, y_pred_lasso)
print('Linear R2: '+ str(r2_lasso))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_lasso = explained_variance_score(y_test, y_pred_lasso)
print('lasso explained variance: '+str(ev_lasso))

Lasso RMSE:1415.4267159088004
Độ chính xác của thuật toán lasso: 0.8549962141499554
Linear R2: 0.8549962141499554
Linear explained variance: 0.8554098197692016


In [ ]:
from sklearn import linear_model
mymodel_Bayes = linear_model.BayesianRidge()
mymodel_Bayes.fit(X_train, y_train)


BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [ ]:
#Đánh giá Bayes model
from sklearn.metrics import mean_squared_error
y_pred_Bayes = mymodel_Bayes.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_Bayes)
Bayes_rmse = np.sqrt(tree_mse)
print("Bayes RMSE:" + str(Bayes_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán Bayes: '+str(mymodel_Bayes.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_Bayes = r2_score(y_test, y_pred_Bayes)
print('Bayes R2: '+ str(r2_Bayes))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Bayes = explained_variance_score(y_test, y_pred_Bayes)
print('Bayes explained variance: '+str(ev_Bayes))

Bayes RMSE:1424.799739705657
Độ chính xác của thuật toán Bayes: 0.8638023456065026
Bayes R2: 0.8638023456065026
Bayes explained variance: 0.863804100603259


In [ ]:
################################### TEST TINH CHINH MO HINH #####################################

In [ ]:

# grid search Ridge model on the auto insurance dataset
from pandas import read_csv
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
# define model
model = Ridge()
# define evaluation
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define search space
space = dict()
space['solver'] = ['svd', 'cholesky', 'lsqr']
space['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
space['fit_intercept'] = [True]
space['normalize'] = [False]
# define search
search = GridSearchCV(model, space, scoring='r2', n_jobs=-1, cv=cv)
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best Score: 0.8600785830654148
Best Hyperparameters: {'alpha': 1e-05, 'fit_intercept': True, 'normalize': False, 'solver': 'svd'}


In [ ]:
from sklearn.metrics import mean_squared_error
y_pred_Ridge = result.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_Ridge)
Ridge_rmse = np.sqrt(tree_mse)
print("Ridge RMSE:" + str(Ridge_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán Ridge: '+str(result.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_Ridge = r2_score(y_test, y_pred_Ridge)
print('Ridge R2: '+ str(r2_Ridge))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Ridge = explained_variance_score(y_test, y_pred_Ridge)
print('Ridge explained variance: '+str(ev_Ridge))

Ridge RMSE:1416.513445757502
Độ chính xác của thuật toán Ridge: 0.865381924842197
Ridge R2: 0.865381924842197
Ridge explained variance: 0.865385305580427


In [ ]:
#GridSearchCV cho linear regression
from pandas import read_csv
from sklearn.linear_model import Ridge
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
model = LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model,parameters, cv=None, scoring='r2')
result=grid.fit(X_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.85508107631206
Best Hyperparameters: {'copy_X': True, 'fit_intercept': True, 'normalize': False}


In [ ]:
from sklearn.linear_model import LinearRegression
model_LinearR = LinearRegression(copy_X=True, fit_intercept=True, normalize=True)
mymodel_LinearR = model_LinearR.fit(X_train, y_train)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(d1, out, random_state=0,test_size=0.25)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred_LinearR = mymodel_LinearR.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_LinearR)
Linear_rmse = np.sqrt(tree_mse)
print("Linear RMSE:" + str(Linear_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán LinearRegression: '+str(mymodel_LinearR.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_Linear = r2_score(y_test, y_pred_LinearR)
print('Linear R2: '+ str(r2_Linear))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Linear = explained_variance_score(y_test, y_pred_LinearR)
print('Linear explained variance: '+str(ev_Linear))

Linear RMSE:2103.2422373523877
Độ chính xác của thuật toán LinearRegression: 0.70321594568779
Linear R2: 0.70321594568779
Linear explained variance: 0.7036292314594204


In [ ]:
###### GridSearchCV DecisionTree ###########
from sklearn.tree import DecisionTreeRegressor
model_DecisionT = DecisionTreeRegressor(max_depth=2, criterion='mae',splitter='best')
mymodel_DecisionT = model_DecisionT.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred_DecisionT = mymodel_DecisionT.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_DecisionT)
tree_rmse = np.sqrt(tree_mse)
print("DecisionTree RMSE: "+str(tree_rmse))

print('Độ chính xác của thuật toán DecisionTree: '+str(mymodel_DecisionT.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_DecisionT = r2_score(y_test, y_pred_DecisionT)
print('Decision R2: '+ str(r2_DecisionT))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Decision = explained_variance_score(y_test, y_pred_DecisionT)
print('Decision explained variance: '+str(ev_Decision))

DecisionTree RMSE: 1395.5453748053437
Độ chính xác của thuật toán DecisionTree: 0.8693378207204853
Decision R2: 0.8693378207204853
Decision explained variance: 0.8693394247380879


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV

model = DecisionTreeRegressor()
# define evaluation
cv = 10
# define search space
space = dict()
space['max_depth'] = [1,2,3,4,5,6,7,8,9,10]
space['criterion'] = ['friedman_mse','mse','mae','poisson']
space['max_leaf_nodes'] = [2,3,4,5,6,7,8,9,10]
# define search
search = GridSearchCV(model, space, scoring='r2', n_jobs=-1, cv=cv)
# execute search
result = search.fit(X_train, y_train)
# summarize result
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

Best Score: 0.9784232370432513
Best Hyperparameters: {'criterion': 'friedman_mse', 'max_depth': 10}


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(d1, out, random_state=0,test_size=0.25)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=1, criterion='friedman_mse')
result=model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred_DecisionT = result.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_DecisionT)
tree_rmse = np.sqrt(tree_mse)
print("DecisionTree RMSE: "+str(tree_rmse))

print('Độ chính xác của thuật toán DecisionTree: '+str(result.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_DecisionT = r2_score(y_test, y_pred_DecisionT)
print('Decision R2: '+ str(r2_DecisionT))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Decision = explained_variance_score(y_test, y_pred_DecisionT)
print('Decision explained variance: '+str(ev_Decision))

DecisionTree RMSE: 1875.4144832794727
Độ chính xác của thuật toán DecisionTree: 0.7640301561273879
Decision R2: 0.7640301561273879
Decision explained variance: 0.7641068352498344


In [ ]:
##############################################################################################################

In [ ]:
#Ridge model
from sklearn.linear_model import Ridge
mymodel_Ridge = Ridge(alpha=1e-2,solver='sag',fit_intercept=False, normalize=True)
mymodel_Ridge.fit(X_train, y_train)

NameError: ignored

In [ ]:
#Đánh giá Ridge
from sklearn.metrics import mean_squared_error
y_pred_Ridge = mymodel_Ridge.predict(X_test)
tree_mse = mean_squared_error(y_test, y_pred_Ridge)
Ridge_rmse = np.sqrt(tree_mse)
print("Ridge RMSE:" + str(Ridge_rmse));
#Dộ chính xác
print('Độ chính xác của thuật toán Ridge: '+str(mymodel_Ridge.score(X_test, y_test)))
#   r2
from sklearn.metrics import r2_score
r2_Ridge = r2_score(y_test, y_pred_Ridge)
print('Ridge R2: '+ str(r2_Ridge))
# explained variance score
from sklearn.metrics import explained_variance_score
ev_Ridge = explained_variance_score(y_test, y_pred_Ridge)
print('Ridge explained variance: '+str(ev_Ridge))

Ridge RMSE:3150.6288966608845
Độ chính xác của thuật toán Ridge: 0.3340272215783401
Ridge R2: 0.3340272215783401
Ridge explained variance: 0.33573075585759304


In [ ]:
############## HẾT ###################